## Avro Bytes
---

In [2]:
:cache 1000
:dep apache-avro

cache: 1000 MiB


In [3]:
use apache_avro::{from_value, to_value, Reader, Schema, Writer};
use apache_avro::types::{Value, Record};
use std::collections::HashMap;

---
### Anti-Serde Bytes:

In [4]:
let avsc_raw = r#"
    {
        "type": "record",
        "name": "db",
        "fields": [
            {"name": "bv", "type": "bytes"},
            {"name": "bvm", "type": {"type": "map", "values": "bytes"}}
        ]
    }
"#;

In [5]:
let mut dat_enc: Vec<u8> =
{
    let avsc = Schema::parse_str(avsc_raw).unwrap();
    let mut writer = Writer::new(&avsc, Vec::new());
    let mut rec = Record::new(writer.schema()).unwrap();

    let bv1 = (10..=32).into_iter().cycle().take(64).collect::<Vec<u8>>();
    let bv2 = (0..8).into_iter().cycle().take(32).collect::<Vec<u8>>();
    let mut bvm = HashMap::<String, Vec<u8>>::new();
    bvm.insert("bv2".into(), bv2);
    rec.put("bv", bv1);
    rec.put("bvm", bvm);
    
    writer.append(rec).unwrap();
    writer.into_inner().unwrap()
};
dat_enc

[79, 98, 106, 1, 4, 20, 97, 118, 114, 111, 46, 99, 111, 100, 101, 99, 8, 110, 117, 108, 108, 22, 97, 118, 114, 111, 46, 115, 99, 104, 101, 109, 97, 246, 1, 123, 34, 116, 121, 112, 101, 34, 58, 34, 114, 101, 99, 111, 114, 100, 34, 44, 34, 110, 97, 109, 101, 34, 58, 34, 100, 98, 34, 44, 34, 102, 105, 101, 108, 100, 115, 34, 58, 91, 123, 34, 110, 97, 109, 101, 34, 58, 34, 98, 118, 34, 44, 34, 116, 121, 112, 101, 34, 58, 34, 98, 121, 116, 101, 115, 34, 125, 44, 123, 34, 110, 97, 109, 101, 34, 58, 34, 98, 118, 109, 34, 44, 34, 116, 121, 112, 101, 34, 58, 123, 34, 116, 121, 112, 101, 34, 58, 34, 109, 97, 112, 34, 44, 34, 118, 97, 108, 117, 101, 115, 34, 58, 34, 98, 121, 116, 101, 115, 34, 125, 125, 93, 125, 0, 42, 212, 200, 249, 209, 124, 37, 68, 224, 148, 189, 250, 234, 144, 33, 210, 2, 210, 1, 128, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 10, 11,

In [6]:
String::from_utf8_lossy(&dat_enc[..])

"Obj\u{1}\u{4}\u{14}avro.codec\u{8}null\u{16}avro.schema�\u{1}{\"type\":\"record\",\"name\":\"db\",\"fields\":[{\"name\":\"bv\",\"type\":\"bytes\"},{\"name\":\"bvm\",\"type\":{\"type\":\"map\",\"values\":\"bytes\"}}]}\0*����|%D�����!�\u{2}�\u{1}�\u{1}\n\u{b}\u{c}\r\u{e}\u{f}\u{10}\u{11}\u{12}\u{13}\u{14}\u{15}\u{16}\u{17}\u{18}\u{19}\u{1a}\u{1b}\u{1c}\u{1d}\u{1e}\u{1f} \n\u{b}\u{c}\r\u{e}\u{f}\u{10}\u{11}\u{12}\u{13}\u{14}\u{15}\u{16}\u{17}\u{18}\u{19}\u{1a}\u{1b}\u{1c}\u{1d}\u{1e}\u{1f} \n\u{b}\u{c}\r\u{e}\u{f}\u{10}\u{11}\u{12}\u{13}\u{14}\u{15}\u{16}\u{17}\u{18}\u{19}\u{1a}\u{1b}\u{2}\u{6}bv2@\0\u{1}\u{2}\u{3}\u{4}\u{5}\u{6}\u{7}\0\u{1}\u{2}\u{3}\u{4}\u{5}\u{6}\u{7}\0\u{1}\u{2}\u{3}\u{4}\u{5}\u{6}\u{7}\0\u{1}\u{2}\u{3}\u{4}\u{5}\u{6}\u{7}\0*����|%D�����!�"

In [7]:
let dat = HashMap::<String, Value>::from_iter({
    match Reader::new(&dat_enc[..]).unwrap().last().unwrap().unwrap() {
        Value::Record(r) => Some(r),
        _ => None
    }.unwrap()
});
dat

{"bvm": Map({"bv2": Bytes([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7])}), "bv": Bytes([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])}

In [8]:
let bv = match &dat["bv"] { Value::Bytes(bv) => Some(bv), _ => None}.unwrap().to_owned();
bv

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

In [9]:
let bvm = match &dat["bvm"] { Value::Map(bvm) => Some(bvm), _ => None }.unwrap().to_owned();
bvm

{"bv2": Bytes([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7])}

In [10]:
let bv2 = match &bvm["bv2"] { Value::Bytes(bv) => Some(bv), _ => None }.unwrap().to_owned();
bv2

[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]